この章では**自然言語処理(Natural Language Processing:NLP)**の一分野である**感情(センチメント)分析(sentiment analysis）**を取り上げる。
極性(polarity)に基づいて文章を分類する。また以下の内容と取り上げる。
- テキストデータのクレンジングと準備
- テキスト文書からの特徴ベクトルの構築
- 映画レビューを肯定的な文と否定的な文に分類する機械学習のモデルのトレーニング
- アウトオブコア学習に基づく大規模なテキストデータセットの処理
- 文章コレクションからカテゴリのトピックを推定する。

## 8.1 IMDbの映画レビューデータセットでのテキスト処理
感情分析は**意見マイニング(opinion mining)**と呼ばれる

### 8.1.1 映画レビューデータセットを取得する。
http://ai.stanford.edu/~amaas/data/sentiment/

### 8.1.2 映画レビューデータセットをより便利なフォーマットに変換する。
映画レビューをpandasのDataFrameオブジェクトに読み込む。(約10分)進捗状況と完了までの推定時間を**PyPrind(Python Progress INDIcator)**パッケージを使用することにより、確認する事ができる。

In [1]:
import pyprind
import pandas as pd
import os
# 'basepath'の値を展開したレビューデータセットのディレクトリに書き換える
basepath = 'C:/Users/zundo/Desktop/aclImdb'
labels = {'pos':1, 'neg':0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index = True)
            pbar.update()
            
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:44


データセットに組み込まれているクラスラベルはソート済みであるため、np.randomサブモジュールのpermutation関数を使って行の順番をシャッフルしたDataFrameオブジェクトを作成する。こうすると8.3章でデータをローカルドライブから直接ストリーミングするときに、トレーニングデータセットとテストデータセットに分割するにに役立つ。

作業を行いやすくすために、ひとまとめにしたうえでシャッフルした映画レビューデータセットをCSVファイルに保存する。

In [2]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index = False, encoding= 'utf-8')

In [3]:
df = pd.read_csv('movie_data.csv', encoding = 'utf-8')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


## 8.2 Bowモデルの紹介
文章や単語などのカテゴリデータは、数値に変換しておく。テキストを数値の特徴ベクトルとして表現できる**BoW(bag-ofWorks)**モデルを紹介する。
1. 文章の集合全体から、たとえば単語という一意な**トークン(token)**からなる**語彙(vacaburary)**を作成する。
1. 各文書での各単語の出現回数を含んだ特徴ベクトルを構築する。

### 8.2.1 単語を特徴ベクトルに変換する

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
# print(bag)

In [5]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [6]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


### 8.2.2 TF-IDFを使って単語の関連性を評価する
テキストデータを解析していると各クラスに分類される複数の文書において、同じ単語が出現する。そういいた頻繁に出現する単語はたいてい意味のある情報を含んでない。ｓこで**TF-IDF(Term Frequency-Inverse Document Frequency)**という手法を使用することで、そういった単語の重みを減らすことができる。

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf = True,
                        norm = 'l2', 
                        smooth_idf = True)

np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


### 8.2.3　テキストデータのクレンジング
不要な文字を取り除くことによって、テキストデータをクレンジング(洗浄）することが最初の重要な手順である。

In [8]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

感情分析に確実に役立つ**顔文字(emoticon)**だけを残し、それ以外の句読点は削除する。

In [11]:
import re
def preprocessor(text):
    # HTMLマークアップを削除
    text = re.sub('[^>]*>','',text) 
    
    # 顔文字を検索しemoticonsに格納
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|p)',text) 
    
    # 単語の一部ではない文字を削除し、テキストを小文字に変換する
    text = (re.sub('[\W]+',' ', text.lower()) + ''.join(emoticons).replace('-',''))
    return text

In [12]:
preprocessor(df.loc[0, 'review'][-50:])

'title brazil not available'

In [13]:
preprocessor("</a>This :) is :( atest :-)!")

'this is atest :):(:)'

In [15]:
df['review'] = df['review'].apply(preprocessor)

### 8.4.2 文章をトークン化する
文章をトークン化する1つの方法は、クレンジングした文章を空白文字(スペース、タブ、改行、リターン、改ページ）で区切り、個々の単語に分割することである。

In [23]:
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

トークン化の便利な手法の1つに、**ワードステミング(word stemming)**がある。これは単語を原型に変換することで、関連する単語を同じ語幹にマッピングすることである。

In [22]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

機械学習をする前に**ストップワードの除去(stop-word removal)**を行う。ストップワードとはあらゆる種類のテキストでみられるごくありふれた単語のことである。

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zundo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [25]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer('a runner likes running and runs a lot')[-10:]
    if w not in stop]

['runner', 'likes', 'running', 'runs', 'lot']

In [ ]:
a